<a href="https://colab.research.google.com/github/Luckysolex/Math-to-code/blob/main/MNIST_Quantization_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TinyML-MNIST-Quantization

In [2]:
import tensorflow as tf
import numpy as np

# Load the MNIST dataset
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the pixel values from [0, 255] to [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0

# Reshape images for the model (add channel dimension)
train_images = train_images[..., np.newaxis]
test_images = test_images[..., np.newaxis]

# Define a simple convolutional neural network (CNN) model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
print("\nTraining the model...")
model.fit(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels))

# Evaluate the model
loss, accuracy = model.evaluate(test_images, test_labels)
print(f"\nTest accuracy: {accuracy:.4f}")

# Save the trained model
model_filename = 'mnist_model.h5'
model.save(model_filename)
print(f"Model saved as '{model_filename}'")

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Training the model...
Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 29s 15ms/step - accuracy: 0.8833 - loss: 0.4174 - val_accuracy: 0.9753 - val_loss: 0.0838
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 15ms/step - accuracy: 0.9763 - loss: 0.0795 - val_accuracy: 0.9793 - val_loss: 0.0690
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 28s 15ms/step - accuracy: 0.9833 - loss: 0.0580 - val_accuracy: 0.9807 - val_loss: 0.0595
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 15ms/step - accuracy: 0.9866 - loss: 0.0454 - val_accuracy: 0.9829 - val_loss: 0.0552
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 28s 15ms/step - accuracy: 0.9885 - loss: 0.0377 - val_accuracy: 0.9831 - val_loss: 0.0523
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9780 - loss: 0.0656



Test accuracy: 0.9831
Model saved as 'mnist_model.h5'


Now that we have a trained `mnist_model.h5` file, we can proceed with the TensorFlow Lite quantization process. This process converts the Keras model into a TensorFlow Lite model with 8-bit integer quantization for efficient deployment on edge devices.

In [3]:
import tensorflow as tf

# Load the saved Keras model
model = tf.keras.models.load_model('mnist_model.h5')

# Convert to a quantized tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT] # triggers the int8 math

# Define a representative dataset for full integer quantization
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(100):
    yield [tf.cast(input_value, tf.float32)]

converter.representative_dataset = representative_data_gen

# Ensure that the model uses only integer operations (optional, but good practice for full integer quantization)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # Or tf.uint8 depending on model input requirements
converter.inference_output_type = tf.int8 # Or tf.uint8

tflite_model_quant = converter.convert()

# Save the quantized model
with open('mnist_model_quant.tflite', 'wb') as f:
  f.write(tflite_model_quant)

print("Quantized TensorFlow Lite model saved as 'mnist_model_quant.tflite'")

Saved artifact at '/tmp/tmp1yj63w_y'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  138121346311056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138121346305488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138121346311248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138121346305680: TensorSpec(shape=(), dtype=tf.resource, name=None)


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Quantized TensorFlow Lite model saved as 'mnist_model_quant.tflite'
